In [9]:
import pyodbc

def get_connection():
    """Establece y devuelve una conexión a la base de datos."""
    try:
        
        connection_string = (
            "Driver={ODBC Driver 17 for SQL Server};"
            "Server=DESKTOP-6RS0ODM\\MSSQLSERVER01;"
            "Database=bar;"
            "Trusted_Connection=yes;"
        )
        conn = pyodbc.connect(connection_string)
        return conn
    except pyodbc.InterfaceError as e:
        print(f"Database connection failed: {str(e)}")
        return None
    except pyodbc.Error as e:
        print(f"An error occurred while connecting to the database: {str(e)}")
        return None


In [10]:
from typing import List
from pydantic import ValidationError
import src.model.schema as models
# from database import get_connection


def fetch_data_from_table(conn, query: str, model):
    """Ejecuta una consulta SQL y devuelve los datos validados en una lista del modelo especificado."""
    data = []
    
    if not conn:
        return data
    
    try:
        with conn.cursor() as cursor:
            cursor.execute(query)
            
            # Obtener nombres de columna
            column_names = [column[0] for column in cursor.description]
            
            rows = cursor.fetchall()
            for row in rows:
                try:
                    # Convertir fila a diccionario
                    row_dict = dict(zip(column_names, row))
                    instance = model(**row_dict)
                    data.append(instance)
                except ValidationError as e:
                    print(f"Error de validación en la fila {row}: {e}")
    except Exception as e:
        print(f"Error al ejecutar la consulta: {e}")
    
    return data

def obtener_usuarios(conn) -> List[models.Usuarios]:
    query = "SELECT id, usuario, rol, nombre FROM usuarios"
    return fetch_data_from_table(conn, query, models.Usuarios)

def obtener_productos(conn) -> List[models.Productos]:
    query = "SELECT id_producto, nombre_producto, referencia, cantidad_actual FROM productos"
    return fetch_data_from_table(conn, query, models.Productos)

def obtener_precios(conn) -> List[models.Precios]:
    query = "SELECT id_precio, id_producto, precioxunidad FROM precios"
    return fetch_data_from_table(conn, query, models.Precios)

def obtener_productos_vendidos(conn) -> List[models.ProductosVendidos]:
    query = "SELECT id_productos_vendidos, id_producto, unidades_vendidas FROM productos_vendidos"
    return fetch_data_from_table(conn, query, models.ProductosVendidos)

def obtener_ventas(conn) -> List[models.Ventas]:
    query = "SELECT id_venta, cliente, fecha, id_productos_vendidos, id FROM ventas"
    return fetch_data_from_table(conn, query, models.Ventas)


In [11]:
# Llamada a cada función de consulta y muestra de resultados
def mostrar_datos():
    conn = get_connection()
    usuarios = obtener_usuarios(conn)
    print("\nUsuarios:")
    for usuario in usuarios:
        print(usuario.json())

    productos = obtener_productos(conn)
    print("\nProductos:")
    for producto in productos:
        print(producto.json())

    precios = obtener_precios(conn)
    print("\nPrecios:")
    for precio in precios:
        print(precio.json())

    productos_vendidos = obtener_productos_vendidos(conn)
    print("\nProductos Vendidos:")
    for producto_vendido in productos_vendidos:
        print(producto_vendido.json())

    ventas = obtener_ventas(conn)
    print("\nVentas:")
    for venta in ventas:
        print(venta.json())

    conn.close
# Ejecutar la función principal
if __name__ == "__main__":
    mostrar_datos()



Usuarios:
{"id":1,"usuario":"ldromeromo","rol":"desarrolladora","nombre":"Laura Daniela Romero Montañez"}
{"id":2,"usuario":"pabernalbo","rol":"administradora","nombre":"Paula Andrea Bernal Ospina"}

Productos:
{"id_producto":1,"nombre_producto":"Cerveza Corona","referencia":"C-001","cantidad_actual":50}
{"id_producto":2,"nombre_producto":"Cerveza Heineken","referencia":"C-002","cantidad_actual":60}
{"id_producto":3,"nombre_producto":"Ron Bacardi","referencia":"L-001","cantidad_actual":20}
{"id_producto":4,"nombre_producto":"Vodka Smirnoff","referencia":"L-002","cantidad_actual":15}
{"id_producto":5,"nombre_producto":"Tequila José Cuervo","referencia":"L-003","cantidad_actual":25}
{"id_producto":6,"nombre_producto":"Whisky Jack Daniels","referencia":"L-004","cantidad_actual":30}
{"id_producto":7,"nombre_producto":"Ginebra Tanqueray","referencia":"L-005","cantidad_actual":18}
{"id_producto":8,"nombre_producto":"Coca Cola","referencia":"NA-001","cantidad_actual":100}
{"id_producto":9,"n

: 